# 卷积过程理解为模板匹配
# 卷积核与特征图的对应
# 池化操作是粗糙化原始图像的过程
# dropout（）：防止过拟合，只在训练阶段用，不在测试阶段用
# 卷积---池化过程就是在进行图像特征拾取
# 特征有顺序性就可以利用CNN，传统的dataframe就不可以。

In [1]:
import torchvision.datasets as dsets

In [2]:
import torchvision.transforms as transforms
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
train_datasets=dsets.MNIST('../data',
                           train=True,
                          transform=transforms.ToTensor(),
                          download=False)
test_datasets=dsets.MNIST('../data',
                           train=False,
                          transform=transforms.ToTensor(),
                          download=False)


In [4]:
batch_size=100
image_size=28
num_classes=10

train_loader=torch.utils.data.DataLoader(dataset=train_datasets,
                                        batch_size=batch_size,
                                        shuffle=True,num_workers=0)
test_loader=torch.utils.data.DataLoader(dataset=test_datasets,
                                        batch_size=batch_size,
                                        shuffle=False,num_workers=0)

In [5]:
indices=range(len(test_datasets))
indice_val=indices[:5000]
indice_test=indices[5000:]

In [6]:
def rightness(x1,y1):
    output=x1.detach().numpy()
    target=y1.detach().numpy()
    output=np.argmax(output,1)
    rightness=0
    for i in range(np.size(output,0)):
        if output[i]-target[i]<0.00001:
            rightness=rightness+1
    rightness=rightness/np.size(output,0)
    return rightness

In [7]:
sample_val=torch.utils.data.sampler.SubsetRandomSampler(indice_val)
sample_test=torch.utils.data.sampler.SubsetRandomSampler(indice_test)

In [8]:
validation_loader=torch.utils.data.DataLoader(dataset=train_datasets,
                                        batch_size=batch_size,
                                        shuffle=False,
                                        sampler=sample_val)
test_loader=torch.utils.data.DataLoader(dataset=test_datasets,
                                        batch_size=batch_size,
                                        shuffle=False,
                                        sampler=sample_test)

# self.conv1=nn.Conv2d(1,4,5,padding=2)
# 1表示输入通道数即灰度为1，RGB为3,256真彩色为256；
# 4表示输出通道数即卷积核的个数；
# 5卷积核窗口大小

# dropout随机删除神经元连接，防止过拟合，提高鲁棒性

In [9]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(1,4,5,padding=2)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(4,8,5,padding=2)
        self.fc1=nn.Linear(image_size//4*image_size//4*8,512)
        self.fc2=nn.Linear(512,num_classes)
    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.pool(x)
        x=F.relu(self.conv2(x))
        x=self.pool(x)
        x=x.view(-1,image_size//4*image_size//4*8)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=self.fc2(x)
        x=F.log_softmax(x)
        return x

In [10]:
num_epochs=2
model=ConvNet()
optimizer=torch.optim.SGD(model.parameters(),lr=0.1)
criterion=torch.nn.CrossEntropyLoss()

In [11]:
for epoch in range(num_epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target=Variable(data),Variable(target)
        optimizer.zero_grad()
        
        model.train()
        output=model(data)
        loss=criterion(output,target)
        
        loss.backward()
        optimizer.step()
        if batch_idx%100==0:
            model.eval()
            val_rights=[]
            for data,target in train_loader:
                    data,target=Variable(data),Variable(target)
                    output=model(data)
                    val=rightness(output,target)
                    print("The rightness in train_datasets=",val)


model.eval()
test_loss=0
correct=0
for data,target in test_loader:
    data,target=Variable(data),Variable(target)
    output=model(data)
    val=rightness(output,target)
    print("The rightness in test_datasets=",val)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


The rightness in train_datasets= 0.73
The rightness in train_datasets= 0.7
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.73
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.64
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.69
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.79
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.75
The rightness in train_datasets= 0.69
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.69
The rightness in train_datasets= 0.77
The rightness in train_datasets= 0.8
The rightness 

The rightness in train_datasets= 0.76
The rightness in train_datasets= 0.69
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.76
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.8
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.74
The rightness in train_datasets= 0.7
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.78
The rightness in train_datasets= 0.7
The rightness in train_datasets= 0.84
The rightness in train_datasets= 0.76
The rightness in train_datasets= 0.74
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.69
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.74
The rightness in train_datasets= 0.69
The rightness i

The rightness in train_datasets= 0.76
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.7
The rightness in train_datasets= 0.64
The rightness in train_datasets= 0.65
The rightness in train_datasets= 0.74
The rightness in train_datasets= 0.67
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.64
The rightness in train_datasets= 0.7
The rightness in train_datasets= 0.76
The rightness in train_datasets= 0.74
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.66
The rightness in train_datasets= 0.71
The rightness in train_datasets= 0.59
The rightness in train_datasets= 0.68
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.72
The rightness in train_datasets= 0.75
The rightness in train_datasets= 0.61
The rightness 

The rightness in train_datasets= 0.86
The rightness in train_datasets= 0.9
The rightness in train_datasets= 0.95
The rightness in train_datasets= 0.9
The rightness in train_datasets= 0.85
The rightness in train_datasets= 0.84
The rightness in train_datasets= 0.76
The rightness in train_datasets= 0.93
The rightness in train_datasets= 0.87
The rightness in train_datasets= 0.85
The rightness in train_datasets= 0.85
The rightness in train_datasets= 0.9
The rightness in train_datasets= 0.87
The rightness in train_datasets= 0.91
The rightness in train_datasets= 0.93
The rightness in train_datasets= 0.87
The rightness in train_datasets= 0.89
The rightness in train_datasets= 0.9
The rightness in train_datasets= 0.88
The rightness in train_datasets= 0.86
The rightness in train_datasets= 0.85
The rightness in train_datasets= 0.85
The rightness in train_datasets= 0.78
The rightness in train_datasets= 0.9
The rightness in train_datasets= 0.89
The rightness in train_datasets= 0.92
The rightness in 

...

The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.96
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.97
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.96
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in t

The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in t

The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.97
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.96
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in trai

The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.96
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.97
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in tra

The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.98
The rightness in train_datasets= 1.0
The rightness in train_datasets= 1.0
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.99
The rightness in train_datasets= 0.98
The rightness in train

In [12]:
val

1.0